<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Tools" data-toc-modified-id="Tools-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Tools</a></span><ul class="toc-item"><li><span><a href="#cxx-script" data-toc-modified-id="cxx-script-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>cxx script</a></span></li><li><span><a href="#Name-demangle" data-toc-modified-id="Name-demangle-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Name demangle</a></span></li></ul></li><li><span><a href="#path" data-toc-modified-id="path-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>path</a></span><ul class="toc-item"><li><span><a href="#string-and-unicode" data-toc-modified-id="string-and-unicode-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>string and unicode</a></span></li><li><span><a href="#path::path()" data-toc-modified-id="path::path()-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>path::path()</a></span></li><li><span><a href="#path::string()" data-toc-modified-id="path::string()-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>path::string()</a></span></li></ul></li><li><span><a href="#Path-decomposition" data-toc-modified-id="Path-decomposition-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Path decomposition</a></span></li><li><span><a href="#directory_entry" data-toc-modified-id="directory_entry-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>directory_entry</a></span></li><li><span><a href="#directory_iterator" data-toc-modified-id="directory_iterator-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>directory_iterator</a></span></li><li><span><a href="#Non-member-function" data-toc-modified-id="Non-member-function-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Non-member function</a></span></li></ul></div>

# Tools

You need g++n(n>=8).

In [1]:
!g++ --version | grep g++

g++ (Ubuntu 8.3.0-6ubuntu1~18.04.1) 8.3.0


## cxx script
To run c++ in cell, you need download a c++ source file and build it.

In [2]:
%%script bash
g++ --version | grep g++
ls cxx.cxx
g++ -std=c++03 cxx.cxx -o cxx && echo cxx installed to current directory

g++ (Ubuntu 8.3.0-6ubuntu1~18.04.1) 8.3.0
cxx.cxx
cxx installed to current directory


## Name demangle

If you want to the `typeid(...).name()` looks more clear. You need to include the `demangle.h`, and call `demangle(typeid(...).name())` instead.

In [3]:
!ls demangle.h

demangle.h


# path

## string and unicode

In [4]:
%%script ./cxx -std=c++2a -lstdc++fs

#include <iostream>
#include <filesystem>
#include <string.h>
#include "demangle.h"

int main() {
    using namespace std;
    namespace fs = std::filesystem;

    const char *ptr = "\U0001F60A";
    cout << R"(  "\U0001F60A" )" << ptr << endl;

    // g++ version <= 8, auto -> char const *
    // g++ version >= 9, auto -> char8_t const *
    auto u8_ptr = u8"\U0001F60A";
    cout << R"(typeof(u8"\U0001F60A") )" << demangle(typeid(u8_ptr).name()) << endl;
    // you would see content of string, for g++ version <= 8
    // you would see a pointer address, for g++ version >= 9
    cout << R"(u8"\U0001F60A" )" << u8_ptr << endl;

    const char16_t *u16_ptr = u"\U0001F60A";
    cout << R"( u"\U0001F60A" )"<< u16_ptr << endl;
    
    const char32_t *u32_ptr = U"\U0001F60A";
    cout << R"( U"\U0001F60A" )" << u32_ptr << endl;

}


  "\U0001F60A" 😊
typeof(u8"\U0001F60A") char const*
u8"\U0001F60A" 😊
 u"\U0001F60A" 0x55651da9948a
 U"\U0001F60A" 0x55651da994a0


## path::path()

In [5]:
%%script ./cxx -std=c++2a -lstdc++fs

#include <iostream>
#include <filesystem>
#include <string.h>
#include "demangle.h"

int main() {
    using namespace std;
    namespace fs = std::filesystem;
    
    // path constructed from char const *
    fs::path path1 { "\U0001F60A" };
    
    // path constructed from u8""
    // u8"" is array of char in c++17
    // u8"" is array of char8_t in c++20
    fs::path path2 { u8"\U0001F60A" };
    
    fs::path path4 { u"ascii" };
    
    // path constructed from u"" (UTF-16)
    // g++ refuse to convert unicode for UTF-16
    //fs::path path5 { u"\U0001F60A" };
    
    // path constructed from U"" (UTF-32)
    fs::path path6 { U"\U0001F60A" };
    
    printf("everything OK!\n");
}


everything OK!


## path::string()

In [6]:
%%script ./cxx -std=c++2a -lstdc++fs

#include <iostream>
#include <filesystem>
#include <string.h>
#include "demangle.h"

int main() {
    using namespace std;
    namespace fs = std::filesystem;

    // constructed from UTF-32
    fs::path path { "\U0001F60A" };
    
    // path can work with std::cout
    cout << "path: " << path << endl;
    
    // convert to multiple-bytes
    cout << "path::string " << path.string() << endl;
    
    // If gcc version <= 8, u8string() will return string
    // This will change in the future, u8string() will return u8string
    // cout << path.u8string() << endl;
    
    // not working, we have no u16cout, u32cout
    // cout << path.u16string() << endl;
    // cout << path.u32string() << endl;

    //printf("everything OK!\n");
}

path: "😊"
path::string 😊


# Path decomposition

In [7]:
%%script ./cxx -std=c++2a -lstdc++fs

#include <iostream>
#include <filesystem>

int main() {
    using namespace std;
    namespace fs = std::filesystem;
    
    fs::path path = fs::current_path();
    path.append("a.b");
    cout << "path" <<  path << endl;
    cout << "path.root_name()" <<  path.root_name() << endl;
    cout << "path.root_path()" <<  path.root_path() << endl;
    cout << "path.relative_path()" << path.relative_path() << endl;
    cout << "path.parent_path()" <<  path.parent_path() << endl;
    cout << "path.filename()" << path.filename() << endl;
    cout << "path.stem()" << path.stem() << endl;
    cout << "path.extension()" << path.extension() << endl;
        
}


path"/home/liangh/projects/blog/JupyterNotebooks/a.b"
path.root_name()""
path.root_path()"/"
path.relative_path()"home/liangh/projects/blog/JupyterNotebooks/a.b"
path.parent_path()"/home/liangh/projects/blog/JupyterNotebooks"
path.filename()"a.b"
path.stem()"a"
path.extension()".b"


# directory_entry

In [8]:
%%script ./cxx -std=c++2a -lstdc++fs

#include <iostream>
#include <filesystem>

int main() {
    using namespace std;
    namespace fs = std::filesystem;
    
    auto path = fs::current_path();
    fs::directory_entry de { path };
    cout << "de.exists()" << de.exists() << endl;
    cout << "de.is_directory()" << de.is_directory() << endl;
    cout << "de.is_regular_file()" << de.is_regular_file() << endl;
    cout << "de.is_symlink()" << de.is_symlink() << endl;
    try {
        cout << de.file_size() << endl;        
    } catch(...) {        
    }
        
}


de.exists()1
de.is_directory()1
de.is_regular_file()0
de.is_symlink()0


# directory_iterator

In [9]:
%%script ./cxx -std=c++2a -lstdc++fs


#include <iostream>
#include <filesystem>

int main() {
    using namespace std;
    namespace fs = std::filesystem;
    
    auto path = fs::current_path();
    fs::directory_iterator di { path };
    // directory_iterator is a iterator
    for(int i = 0; di != fs::directory_iterator() && i < 2; ++di, ++i) {
        cout << (*di).path() << endl;
    }
 
    // directory_iterator is a range
    auto i = 0;
    auto rng = fs::directory_iterator(path);
    for(auto const &de : rng) {
        // de: const &directory_entry
        cout << de.path() << endl;

        ++i;
        if(i >=2) break;
    }
    i = 0;
    for(auto const &de : rng) {
        // de: const &directory_entry
        cout << de.path() << endl;

        ++i;
        if(i >=2) break;
    }
}


"/home/liangh/projects/blog/JupyterNotebooks/cxx_test"
"/home/liangh/projects/blog/JupyterNotebooks/BiasAndVarianceInPolyFit.ipynb"
"/home/liangh/projects/blog/JupyterNotebooks/cxx_test"
"/home/liangh/projects/blog/JupyterNotebooks/BiasAndVarianceInPolyFit.ipynb"
"/home/liangh/projects/blog/JupyterNotebooks/BiasAndVarianceInPolyFit.ipynb"
"/home/liangh/projects/blog/JupyterNotebooks/structure_binding_tuple.cxx"


# Non-member function

In [10]:
%%script ./cxx -std=c++2a -lstdc++fs

#include <iostream>
#include <filesystem>


void my_remove_all(std::filesystem::path const &path) {
    using namespace std;
    std::error_code ec;
    std::filesystem::remove_all(path, ec);
    if(ec) {
        cout << "can't remove " << path << " because " 
            << ec.message() << endl;
    }
}

int main() {
    using namespace std;
    namespace fs = std::filesystem;

    cout << "fs::current_path() " << fs::current_path() << endl;
    cout << fs::is_directory(fs::current_path()) << endl;
    cout << "available " << fs::space(fs::current_path()).available/(1024*1024*1024) << "G" << endl;
    
    fs::path path { "." };
    auto parent_path = fs::canonical(path).parent_path();
    cout << fs::absolute(path) << endl;
    cout << fs::canonical(path) << endl;
    cout << fs::weakly_canonical(path) << endl;
    cout << (parent_path) << endl;
    cout << fs::relative(parent_path) << endl;
    
    my_remove_all(fs::path{"a"});
    my_remove_all(fs::path{"b"});

    fs::create_directory(fs::path{"a"});
    fs::create_directory(fs::path{"a/1"});
    fs::copy(fs::path{"a"}, fs::path{"b"}, fs::copy_options::recursive);
    cout << exists(fs::path{"b/1"}) << endl;
    fs::rename(fs::path{"b/1"}, fs::path{"b/2"});
    fs::remove(fs::path{"b/2"});
    
    cout << exists(fs::path{"b/1"}) << exists(fs::path{"b/2"}) << endl;    
    
    my_remove_all(fs::path{"a"});
    my_remove_all(fs::path{"b"});
    
    
}


fs::current_path() "/home/liangh/projects/blog/JupyterNotebooks"
1
available 1864G
"/home/liangh/projects/blog/JupyterNotebooks/."
"/home/liangh/projects/blog/JupyterNotebooks"
"/home/liangh/projects/blog/JupyterNotebooks"
"/home/liangh/projects/blog"
".."
1
00
